# My Own Learner
> Notebook for exercises making my own learner class.

- toc: true 
- badges: true
- comments: true
- categories: [jupyter]


In [17]:
import fastbook
fastbook.setup_book()

from fastbook import *
from fastai.vision.widgets import *

In [18]:
## MyLearner class

In [19]:
class MyLearner():

    def __init__(self, dls : DataLoaders, model, loss_func, opt_func, metrics, lr=1e-3):
        self.dls = dls
        self.model = model
        self.loss_func = loss_func
        self.opt_func = opt_func
        self.metrics = metrics
        self.lr = lr
        self.opt = self.opt_func([p for p in self.model.parameters() if p.requires_grad], lr=self.lr)

    def fit(self, n_epoch):
        for i in range(n_epoch):
            self.train_epoch()
            print(f"t_loss: {round(float(self.loss), 4)} acc: {self.validate_epoch()}", end='\n')

    def train_epoch(self):
        for xb,yb in self.dls.train:
            self.calc_grad(xb, yb)
            self.opt.step()
            self.opt.zero_grad()

    def calc_grad(self, xb, yb):
        self.preds = self.model(xb)
        self.loss_grad = self.loss_func(self.preds, yb)
        self.loss = self.loss_grad.clone()
        self.loss_grad.backward()


    def validate_epoch(self):
        accs = [self.metrics(self.model(xb), yb) for xb,yb in self.dls.valid]
        return round(torch.stack(accs).mean().item(), 4)

In [20]:
class BasicOptim:
    def __init__(self, params, lr): 
        self.params = params
        self.lr = lr
        
    def step(self, *args, **kwargs):
        for p in self.params:
            p.data -= p.grad.data * self.lr
    
    def zero_grad(self, *args, **kwargs):
        for p in self.params: p.grad = None

In [21]:
## Models

In [22]:
# Linear model
def linear1(xb): return xb@weights + bias

In [23]:
# Simple neural network
def simple_net(xb):
    res = xb@w1 + b1
    res = res.max(tensor(0.0))
    res = res@w2 + b2
    return res

In [24]:
## Test MyLearner class

In [25]:
# Load in training data
path = untar_data(URLs.MNIST_SAMPLE)
Path.BASE_PATH = path
threes = (path/'train'/'3').ls().sorted()
sevens = (path/'train'/'7').ls().sorted()

In [26]:
seven_tensors = [tensor(Image.open(o)) for o in sevens]
three_tensors = [tensor(Image.open(o)) for o in threes]

In [27]:
stacked_sevens = torch.stack(seven_tensors).float()/255
stacked_threes = torch.stack(three_tensors).float()/255

In [28]:
train_x = torch.cat([stacked_threes, stacked_sevens]).view(-1, 28*28)
train_y = tensor([1]*len(threes) + [0]*len(sevens)).unsqueeze(1)

In [29]:
#Package training data into data loader
dataSet = list(zip(train_x,train_y))
dataLoad = DataLoader(dataSet, batch_size=256)

In [30]:
valid_3_tens = torch.stack([tensor(Image.open(o))
    for o in (path/'valid'/'3').ls()])
valid_3_tens = valid_3_tens.float()/255
valid_7_tens = torch.stack([tensor(Image.open(o))
        for o in (path/'valid'/'7').ls()])
valid_7_tens = valid_7_tens.float()/255

In [31]:
valid_x = torch.cat([valid_3_tens, valid_7_tens]).view(-1, 28*28)
valid_y = tensor([1]*len(valid_3_tens) + [0]*len(valid_7_tens)).unsqueeze(1)
valid_dset = list(zip(valid_x,valid_y))
validDataLoad = DataLoader(valid_dset, batch_size=256)

In [32]:
batch = train_x[:4]
batch.shape

torch.Size([4, 784])

In [33]:
preds = linear1(batch)
loss = learner.mnist_loss(preds, train_y[:4])
loss

NameError: name 'weights' is not defined

In [ ]:
loss.backward()
weights.grad.shape,weights.grad.mean(),bias.grad

In [ ]:
learner = MyLearner()

learner.calc_grad(batch, train_y[:4], linear1)
weights.grad.mean(),bias.grad

In [ ]:
weights.grad.zero_()
bias.grad.zero_()

In [ ]:
learner.batch_accuracy(linear1(train_x[:4]), train_y[:4])

In [ ]:
learner.validate_epoch(linear1, validDataLoad)

In [ ]:
lr = 1.
params = weights,bias
learner.train_epoch(linear1, lr, params, dataLoad)
learner.validate_epoch(linear1, validDataLoad)

In [ ]:
for i in range(20):
    learner.train_epoch(linear1, lr, params, dataLoad)
    print(learner.validate_epoch(linear1, validDataLoad), end=' ')

In [ ]:
images, targes = dataLoad.dataset[1]
len(images)

In [34]:
def mnist_loss(predictions, targets):
    predictions = predictions.sigmoid()
    return torch.where(targets==1, 1-predictions, predictions).mean()

In [35]:
def batch_accuracy(xb, yb):
    preds = xb.sigmoid()
    correct = (preds>0.5) == yb
    return correct.float().mean()

In [36]:
my_net = nn.Sequential(
    nn.Linear(28*28,128),
    nn.ReLU(),
    nn.Linear(128,64),
    nn.ReLU(),
    nn.Linear(64,1),
)


In [37]:
dls = DataLoaders(dataLoad, validDataLoad)

my_learner = MyLearner(dls, my_net, loss_func=mnist_loss, opt_func=SGD, metrics=batch_accuracy)

my_learner.fit(40)

t_loss: 0.5025 acc: 0.3043
t_loss: 0.5019 acc: 0.3547
t_loss: 0.5013 acc: 0.4003
t_loss: 0.5007 acc: 0.4582
t_loss: 0.5001 acc: 0.5201
t_loss: 0.4995 acc: 0.5902
t_loss: 0.4988 acc: 0.6499
t_loss: 0.4982 acc: 0.7001
t_loss: 0.4975 acc: 0.7464
t_loss: 0.4968 acc: 0.7818
t_loss: 0.4961 acc: 0.8147
t_loss: 0.4954 acc: 0.8407
t_loss: 0.4947 acc: 0.8623
t_loss: 0.4939 acc: 0.8731
t_loss: 0.4931 acc: 0.885
t_loss: 0.4923 acc: 0.8968
t_loss: 0.4915 acc: 0.9057
t_loss: 0.4906 acc: 0.9111
t_loss: 0.4897 acc: 0.9174
t_loss: 0.4887 acc: 0.9234
t_loss: 0.4877 acc: 0.9259
t_loss: 0.4867 acc: 0.9288
t_loss: 0.4856 acc: 0.9347
t_loss: 0.4846 acc: 0.9371
t_loss: 0.4834 acc: 0.9391
t_loss: 0.4822 acc: 0.9416
t_loss: 0.481 acc: 0.9416
t_loss: 0.4797 acc: 0.9426
t_loss: 0.4784 acc: 0.9426
t_loss: 0.4771 acc: 0.9446
t_loss: 0.4756 acc: 0.9441
t_loss: 0.4742 acc: 0.9451
t_loss: 0.4727 acc: 0.9461
t_loss: 0.4711 acc: 0.9466
t_loss: 0.4694 acc: 0.9466
t_loss: 0.4677 acc: 0.9466
t_loss: 0.466 acc: 0.9475
t_lo